In [41]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [4]:
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt > input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  6989k      0 --:--:-- --:--:-- --:--:-- 7027k


In [5]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [10]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [97]:
# data loading
def get_batch(split, batch_size=4, block_size=8):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model, eval_iters):
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(split)
            _, loss = model(xb, targets=yb)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    
block_size = 8
batch_size = 4
xb, yb = get_batch('train', batch_size=4, block_size=block_size)
print(xb.shape, yb.shape)

torch.Size([4, 8]) torch.Size([4, 8])


In [68]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size=27, n_embed=10):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape # idx ranges from 0~vocab_size
        tok_embed = self.token_embedding_table(idx) # B, T, C
        logits = self.lm_head(tok_embed) # B, T, vocab_size

        if targets is None:
            loss = None
        else: # assuming targets has dimension (B, T)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
            
    def generate(self, idx, max_new_tokens):
        # idx is B, T size
        for _ in range(max_new_tokens):
            logits, _ = self(idx[:, -1:]) # Only focus on last time stamp: B, 1, vocab_size
            probs = F.softmax(logits, dim=-1) # B, 1, vocab_size
            probs = probs[:, -1, :] # B, vocab_size
            idx_next = torch.multinomial(probs, num_samples=1) # B, 1
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
        
m = BigramLanguageModel(vocab_size, n_embed=16)
out, loss = m(xb, yb)
print(out.shape, loss)

torch.Size([32, 65]) tensor(4.3888, grad_fn=<NllLossBackward0>)


In [99]:
print(estimate_loss(model, eval_iters=10))

{'train': tensor(2.4845), 'val': tensor(2.5985)}


In [81]:
idx_out = m.generate(idx=torch.zeros(2, 1, dtype=torch.long), max_new_tokens=100)
print(decode(idx_out[0].tolist()))


kg
c,'ywjZ xdjHZTiv;xdbX'
uMnVWr 'uBq$&rvM$
fGlTWx:IvNF!;y-ztl;FRYHr stHE-Y,SpMYxpSEaXlF.oT3zSfEqLa3


In [102]:
torch.manual_seed(1337)
model = BigramLanguageModel(vocab_size, n_embed=16)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2)

batch_size = 32

for steps in range(20000):
    xb, yb = get_batch("train", batch_size=batch_size)
    logits, loss = model(xb, targets=yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if steps % 1000 == 0:
        print(estimate_loss(model, eval_iters=20))

{'train': tensor(4.3036), 'val': tensor(4.2983)}
{'train': tensor(2.4417), 'val': tensor(2.4879)}
{'train': tensor(2.3617), 'val': tensor(2.4146)}
{'train': tensor(2.5253), 'val': tensor(2.5032)}
{'train': tensor(2.4525), 'val': tensor(2.4584)}
{'train': tensor(2.5602), 'val': tensor(2.5186)}
{'train': tensor(2.5137), 'val': tensor(2.4933)}
{'train': tensor(2.4191), 'val': tensor(2.4185)}
{'train': tensor(2.4012), 'val': tensor(2.4168)}
{'train': tensor(2.5034), 'val': tensor(2.4503)}
{'train': tensor(2.4425), 'val': tensor(2.5656)}
{'train': tensor(2.5026), 'val': tensor(2.4457)}
{'train': tensor(2.4930), 'val': tensor(2.4619)}
{'train': tensor(2.4090), 'val': tensor(2.4638)}
{'train': tensor(2.5540), 'val': tensor(2.5224)}
{'train': tensor(2.4811), 'val': tensor(2.5075)}
{'train': tensor(2.5268), 'val': tensor(2.4874)}
{'train': tensor(2.4796), 'val': tensor(2.4628)}
{'train': tensor(2.5198), 'val': tensor(2.5007)}
{'train': tensor(2.4336), 'val': tensor(2.5210)}


In [96]:
idx_out = model.generate(idx=torch.zeros(2, 1, dtype=torch.long), max_new_tokens=500)
print(decode(idx_out[0].tolist()))


GH: ofrs'st mst!
Pin GLo cohesinse betizlerk, bus the
I ngoss.
I:
hthe our sthore y ooutheswh w loorm, ice hit heg, I r mathtavoy uaswllit wooferestithexthovould thes mitndrleaisstout fisidve choures;


Fie mmeppp-HES:
Lo mest ios t demase 'd id:
NBUFoner, thexpeacitt.
TESase
Wepe
Anou ooundels we
Thshe 'deerdex's ny. alind te,
K:

ARDue;
Fowse be h wel MNulde.
Whoussie t this
I:
OMy ivesto, ar d m ctha Conequs dlod; ias d hicongl, bl ilathrer'din, wer s,
Ang ope theemalplowhentus thecood ngswno


In [ ]:
class Head(nn.Module):
    def __init__(self, n_embed=10, block_size=8, head_size=16, dropout=0.2):
        super().__init__()
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones((block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
                             
    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x) # B, T, H
        k = self.key(x) # B, T, H
        v = self.value(x)
        wei = q @ k.transpose(-2, -1) * head_size**-0.5 # B, T, T
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) # B, T, T
        wei = F.soft_max(wei / torch.sqrt(head_size), dim=-1)
        wei = self.dropout(wei)
        self.out = wei @ v
        return self.out

class 